In [1]:
def cool(ref1,ref2,mf1,mf2,out,in1,valin1,in2,valin2):
    import CoolProp.CoolProp as CP
    from CoolProp.CoolProp import PropsSI
    if ref2==0:
        k='HEOS::'+ref1
        g=CP.PropsSI(out,in1,valin1,in2,valin2,k)
        return g
    else:
        a=ref1+'&'+ref2
        b=CP.AbstractState('HEOS',a);b.set_mass_fractions([mf1,mf2])
        c=b.get_mole_fractions()
        k='HEOS::'+ref1+'['+'{}'.format(c[0])+']&'+ref2+'['+'{}'.format(c[1])+']'
        g=CP.PropsSI(out,in1,valin1,in2,valin2,k)
        return g

In [2]:
import CoolProp.CoolProp as CP
import matplotlib.pyplot as plt
import pandas as pd
import math
import numpy as np
from CoolProp.CoolProp import PropsSI
#input data
T1=543;T7=T8=463;P4=45000;T13=298;T11=T7+30;B=60;b=0.0022;pco=0.01325;we=0.55;g=9.81
#EXHAUST INPUTS
T9=587;mex=105.5;cex=1048.2;dex=0.59508;hex=593967;vex=.000030132;kex=0.045794
#MATERIAL PROPERTIES
t=0.0005;k=17
#COMMAN PARAMETER
X=(b*math.pi)/pco
si=(1/6)*(1+((1+X**2)**0.5)+4*(1+((X**2)/2))**0.5)
dh=(2*b)/si
deq=2*b
Gex=(mex)/(b*we)



#REQUIRED COMMAN PROPERTIES OF EVAPORATOR
P8=cool('Benzene',0,0,0,'P','T',T8,'Q',1)
cge=cool('Benzene',0,0,0,'C','T',T8,'Q',1)
cle=cool('Benzene',0,0,0,'C','T',T8,'Q',0)
kle=cool('Benzene',0,0,0,'L','T',T8,'Q',0)
kge=cool('Benzene',0,0,0,'L','T',T8,'Q',1)
vge=cool('Benzene',0,0,0,'V','T',T8,'Q',1)
vle=cool('Benzene',0,0,0,'V','T',T8,'Q',0)
prle=cool('Benzene',0,0,0,'Prandtl','T',T8,'Q',0)
prge=cool('Benzene',0,0,0,'Prandtl','T',T8,'Q',1)
dle=cool('Benzene',0,0,0,'D','T',T8,'Q',0)
dge=cool('Benzene',0,0,0,'D','T',T8,'Q',1)
s_tension=cool('Benzene',0,0,0,'I','T',T8,'Q',1)
hge=cool('Benzene',0,0,0,'H','T',T8,'Q',1)
hle=cool('Benzene',0,0,0,'H','T',T8,'Q',0)
STle=cool('Benzene',0,0,0,'I','T',T8,'Q',0)
STge=cool('Benzene',0,0,0,'I','T',T8,'Q',1)
LHE=hge-hle

#REQUIRED COMMAN PROPERTIES OF CONDENSER
cgc=cool('Benzene',0,0,0,'C','P',P4,'Q',1)
clc=cool('Benzene',0,0,0,'C','P',P4,'Q',0)
klc=cool('Benzene',0,0,0,'L','P',P4,'Q',0)
vgc=cool('Benzene',0,0,0,'V','P',P4,'Q',1)
vlc=cool('Benzene',0,0,0,'V','P',P4,'Q',0)
prlc=cool('Benzene',0,0,0,'Prandtl','P',P4,'Q',0)
dlc=cool('Benzene',0,0,0,'D','P',P4,'Q',0)
dgc=cool('Benzene',0,0,0,'D','P',P4,'Q',1)
hgc=cool('Benzene',0,0,0,'H','P',P4,'Q',1)
hlc=cool('Benzene',0,0,0,'H','P',P4,'Q',0)
LHC=hgc-hlc
STlc=cool('Benzene',0,0,0,'I','P',P4,'Q',0)
STgc=cool('Benzene',0,0,0,'I','P',P4,'Q',1)

#REQUIRED COMMAN PROPERTIES OF WATER
cw=cool('water',0,0,0,'C','P',101325,'T',T13)
kw=cool('water',0,0,0,'L','P',101325,'T',T13)
vw=cool('Benzene',0,0,0,'V','P',101325,'T',T13)
prw=cool('Benzene',0,0,0,'Prandtl','P',101325,'T',298)

#UNKNOWN TEMPRETURS ,PRESSURE & OTHER PROPERTIES
T4=cool('Benzene',0,0,0,'T','P',P4,'Q',0);T3=T4
T14=T3-10

P7=cool('Benzene',0,0,0,'P','T',T7,'Q',0)
S1=cool('Benzene',0,0,0,'S','T',T1,'P',P7)
S2=S1
P2=P4
T2=cool('Benzene',0,0,0,'T','S',S1,'P',P2)
win=(1/dlc)*(P7-P4)
h5=hlc+win
T5=cool('Benzene',0,0,0,'T','H',h5,'P',P7)
T6=T5+(cge/cle)*(T2-T3)


#mass flow rate of touelene
mt=(mex*cex*(T9-T11))/(cge*(T1-T8)+LHE)
print('mt',mt)
T12=T11-((mt*cle)/(mex*cex))*(T7-T6)
T10=((mt*LHE)/(mex*cex))+T11
#MASS FLOW RATE OF WATER
mw=(mt*LHC)/(cw*(T14-T13))
print('mw',mw)


#EQUIVALENT PROPERTIES FOR CONDENSER
x=0
for x in [np.arange(x,1,0.05)]:
    G=(mt)/(b*we)
    Geqc=G*((1-x)+x*((dlc/dgc)**0.5))
    Reeqc=(Geqc*dh)/vlc

#EQUIVALENT PROPERTIES OF EVAPORATER
    Geqe=G*((1-x)+x*((dle/dge)**0.5))
    Reeqe=(Geqe*dh)/vle


mt 23.17743695523348
mw 108.64506197399396


In [3]:
print("vle/vge",vle/vge) # <1000...hence use friedel pr. drop for evaporator

print("vlc/vgc",vlc/vgc) #<1000...hence use friedel pr. drop for condenser

vle/vge 11.03453030547248
vlc/vgc 49.52347678169304


In [4]:
#Evaporator pressure drop

x=0
for x in [np.arange(x,1,0.05)]:
    #mtv=mt*(dle+dge)
    #Friction
    rho_h=((x/dge)+((1-x)/dle))**(-1)
    mtv=mt*rho_h
    #print(mtv,mt*rho_h)
    WeL=(mtv**2)*dh/(rho_h*STle)
    H=((dle/dge)**0.91)*((vge/vle)**0.19)*((1-(vge/vle))**0.7)
    F=(x**0.78)*((1-x)**0.224)
    Rel=mtv*dh/vle
    Reg=mtv*dh/vge
    fle=0.079/Rel
    fge=0.079/Reg
    E=((1-x)**2)+((x**2)*dle*fge/(dge*fle))
    Frh=(mtv**2)/(9.81*dh*(rho_h**2))
    phi_Lo2=E+(3.24*F*H/((Frh**0.045)*(WeL**0.035)))
    del_PL=4*fle*(1/dh)*(mtv**2)*(1/(2*dle))
    del_Pf=del_PL*phi_Lo2*3
    
    #Momentum
    VF=(x/dge)*((((1+0.12*(1-x))*((x/dge)+((1-x)/dle)))+(1.18*(1-x)*((9.81*STle*(dle-dge))**0.25)/(mtv*mtv*(dle**(0.5)))))**(-1))
    del_Pm=mtv*mtv*((1/(dge*VF))-(1/(dle*(1-VF))))
       
    del_P=del_Pf+del_Pm
    
    data={ ' quality  ': x,'del_P':del_P}
    df=pd.DataFrame(data=data)
    print(df)

del_P=del_Pm[19]+del_Pf[19]
P8new=P8+del_P
T8new=PropsSI('T','P',P8new,'Q',1,'Benzene')
P8=PropsSI('P','T',463,'Q',1,'Benzene')
print(P8,P8new,T8new)

     quality           del_P
0         0.00           inf
1         0.05  3.766099e+06
2         0.10  1.188479e+06
3         0.15  5.651007e+05
4         0.20  3.226525e+05
5         0.25  2.043822e+05
6         0.30  1.382352e+05
7         0.35  9.762270e+04
8         0.40  7.089683e+04
9         0.45  5.230422e+04
10        0.50  3.873109e+04
11        0.55  2.835819e+04
12        0.60  2.003991e+04
13        0.65  1.298507e+04
14        0.70  6.564280e+03
15        0.75  1.491439e+02
16        0.80 -7.118979e+03
17        0.85 -1.689825e+04
18        0.90 -3.381731e+04
19        0.95 -8.054662e+04
1216743.6658538124 1136197.046442405 458.9633136082929


c:\users\lenovo\desktop\chinu g\lol\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide


In [5]:
# Condenser

x=0
for x in [np.arange(x,1,0.05)]:
    #mtv=mt*(dle+dge)
    #Friction
    rho_h=((x/dgc)+((1-x)/dlc))**(-1)
    mtv=mt*rho_h
    #print(mtv,mt*rho_h)
    WeL=(mtv**2)*dh/(rho_h*STlc)
    H=((dlc/dgc)**0.91)*((vgc/vlc)**0.19)*((1-(vgc/vlc))**0.7)
    F=(x**0.78)*((1-x)**0.224)
    Rel=mtv*dh/vlc
    Reg=mtv*dh/vgc
    flc=0.079/Rel
    fgc=0.079/Reg
    E=((1-x)**2)+((x**2)*dlc*fgc/(dgc*flc))
    Frh=(mtv**2)/(9.81*dh*(rho_h**2))
    phi_Lo2=E+(3.24*F*H/((Frh**0.045)*(WeL**0.035)))
    del_PL=4*flc*(1/dh)*(mtv**2)*(1/(2*dlc))
    del_Pf=del_PL*phi_Lo2*10
    
    #Momentum
    VF=(x/dgc)*((((1+0.12*(1-x))*((x/dgc)+((1-x)/dlc)))+(1.18*(1-x)*((9.81*STlc*(dlc-dgc))**0.25)/(mtv*mtv*(dlc**(0.5)))))**(-1))
    del_Pm=mtv*mtv*((1/(dgc*VF))-(1/(dlc*(1-VF))))
       
    del_P=del_Pf+del_Pm
    
    data={ ' quality  ': x,'del_P':del_P}
    df=pd.DataFrame(data=data)
    print(df)

del_P=del_Pm[19]+del_Pf[19]
P3=P4
P3new=P3+del_P
T3new=PropsSI('T','P',P3new,'Q',1,'Benzene')
T3=PropsSI('T','P',P3,'Q',1,'Benzene')
print(P3,P3new,T3new)

     quality           del_P
0         0.00           inf
1         0.05  3.019820e+05
2         0.10  7.653128e+04
3         0.15  3.425294e+04
4         0.20  1.940337e+04
5         0.25  1.252003e+04
6         0.30  8.777302e+03
7         0.35  6.518067e+03
8         0.40  5.049351e+03
9         0.45  4.039883e+03
10        0.50  3.315087e+03
11        0.55  2.775816e+03
12        0.60  2.362288e+03
13        0.65  2.036577e+03
14        0.70  1.773462e+03
15        0.75  1.555240e+03
16        0.80  1.368399e+03
17        0.85  1.200566e+03
18        0.90  1.034440e+03
19        0.95  8.139991e+02
45000 45813.999141525994 329.4815081907921


c:\users\lenovo\desktop\chinu g\lol\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in true_divide


In [6]:
#Evaporator Motinski heat transfer coefficient correlation

h8new=PropsSI('H','T',T8new,'Q',1,'Benzene')
h7=PropsSI('H','T',T7,'Q',0,'Benzene')
LHEnew=h8new-h7
Pcrit=CP.PropsSI("Pcrit","Benzene")
q=(mt*LHEnew)/(deq*we)
Pratio=P8new/Pcrit
Fp=(1.8*(Pratio**0.17))+(4*(Pratio**1.2))+(10*(Pratio**10))
he_m=0.00417*Fp*(Pcrit**0.69)*(q**0.7)/10e4
print("he_m",he_m)

he_m 14750.49748396708


In [7]:
# Area of evaporator

#OVERALL HEAT TRANSFER COEFFICIENT
U=((1/hex)+(1/(he_m*1000))+0.0009)**-1

#LMTD
LMTD=((T10-T8new)-(T11-T7))/(math.log((T10-T8new)/(T11-T7)))

#HEAT TRANSFER
Q=mt*LHEnew

#AREA
A_EVA=(Q)/(LMTD*U)

print("A_EVA",A_EVA)



A_EVA 107.03004744810504


In [8]:
#Condenser Motinski heat transfer coefficient correlation

h3new=PropsSI('H','T',T3new,'Q',1,'Benzene')
h4=PropsSI('H','T',T4,'Q',0,'Benzene')
LHCnew=h3new-h4
Pcrit=CP.PropsSI("Pcrit","Benzene")
q=(mt*LHCnew)/(deq*we)
Pratio=P3new/Pcrit
Fp=(1.8*(Pratio**0.17))+(4*(Pratio**1.2))+(10*(Pratio**10))
hc_m=0.00417*Fp*(Pcrit**0.69)*(q**0.7)/10e4
print("hc_m",hc_m)

# FIND HEAT TRANSFER COEFFICIENT FROM WATER SIDE
Gw=(mw)/(b*we)
Rew=(Gw*deq)/vw
h_water=(0.724*(kw/dh)*(((6*B)/math.pi)**0.646)*(Rew**0.583)*(prw**0.33))
print('h_water',h_water)

# Area of condenser

Q=mt*LHCnew
print('Q',Q)
#3.LMTD
LMTD=((T4-T13)-(T3new-T14))/(math.log((T4-T13)/(T3new-T14)))
print('LMTD',LMTD)
#4. OVERALL HEAT TRANSFER COEFFICIENT
U=((1/(hc_m*22000))+(1/h_water)+0.00045)**-1
print('U',U)
#5.Area
A_cond=(Q)/(U*LMTD)
print('A_cond',A_cond)

hc_m 7437.722160087973
h_water 10826331.799360985
Q 9547543.756500721
LMTD 18.926244496084983
U 2221.7360138500203
A_cond 227.0569272954118
